In [ ]:
import lerobot
from lerobot.policies.factory import make_policy
from lerobot.configs.train import TrainPipelineConfig, PreTrainedConfig
from lerobot.policies.pi05 import (  # noqa: E402
    PI05Config,
    PI05Policy,
    make_pi05_pre_post_processors,  # noqa: E402
    
)
from lerobot.configs.default import DatasetConfig, EvalConfig, WandBConfig
from pprint import pprint


In [ ]:
import gc

if "policy" in locals():
    del policy
gc.collect()

# Mock data

In [ ]:
config = PI05Config(max_action_dim=7, max_state_dim=14, dtype="bfloat16", device="cuda")

    # Set up input_features and output_features in the config
from lerobot.configs.types import FeatureType, PolicyFeature

config.input_features = {
    "observation.state": PolicyFeature(
        type=FeatureType.STATE,
        shape=(14,),
    ),
    "observation.images.base_0_rgb": PolicyFeature(
        type=FeatureType.VISUAL,
        shape=(3, 224, 224),
    ),
}

config.output_features = {
    "action": PolicyFeature(
        type=FeatureType.ACTION,
        shape=(7,),
    ),
}

In [ ]:
policy = PI05Policy(config)


In [ ]:
import torch

batch_size = 1
device = "cuda"
dataset_stats = {
        "observation.state": {
            "mean": torch.zeros(14),
            "std": torch.ones(14),
            "min": torch.zeros(14),
            "max": torch.ones(14),
            "q01": torch.zeros(14),
            "q99": torch.ones(14),
        },
        "action": {
            "mean": torch.zeros(7),
            "std": torch.ones(7),
            "min": torch.zeros(7),
            "max": torch.ones(7),
            "q01": torch.zeros(7),
            "q99": torch.ones(7),
        },
        "observation.images.base_0_rgb": {
            "mean": torch.zeros(3, 224, 224),
            "std": torch.ones(3, 224, 224),
            "q01": torch.zeros(3, 224, 224),
            "q99": torch.ones(3, 224, 224),
        },
    }
preprocessor, postprocessor = make_pi05_pre_post_processors(config=config, dataset_stats=dataset_stats)
batch = {
        "observation.state": torch.randn(batch_size, 14, dtype=torch.float32, device=device),
        "action": torch.randn(batch_size, config.chunk_size, 7, dtype=torch.float32, device=device),
        "observation.images.base_0_rgb": torch.rand(
            batch_size, 3, 224, 224, dtype=torch.float32, device=device
        ),  # Use rand for [0,1] range
        "task": ["Pick up the object"] * batch_size,
    }

In [ ]:
input = preprocessor(batch)
input

In [ ]:
action = policy.select_action(input)
output = postprocessor(action)
output

# Xarm

In [ ]:
from lerobot.datasets.lerobot_dataset import LeRobotDataset, LeRobotDatasetMetadata

dataset_path= "/media/ssd/eugene/robotic_manipulation/lerobot_tests/datasets/lite6_record_scripted_250622"
